<a href="https://colab.research.google.com/github/ribesstefano/chalmers_dat450_ml_for_nlp/blob/main/A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook could be found at this link: https://colab.research.google.com/drive/1f2nRQyfELRy0H8q0i5dFrVinJeuiTvz9?usp=sharing

Group N - Authors:

* Stefano Ribes
* Cody Hesse
* Apoorva x

# Assignment 2: Topic Modeling

## Setup and Imports

In [93]:
import numpy as np
import pandas as pd
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('reuters')
nltk.download('brown')
from nltk.corpus import reuters, brown, stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from tqdm import tqdm

from sklearn.datasets import fetch_20newsgroups

import spacy
nlp = spacy.load("en_core_web_sm")

import itertools

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


## Corpus Download and Cleaning

In [18]:
def tokenize(text, lowercase=True):
    if lowercase:
        return [t.text.lower() for t in nlp.tokenizer(text)]
    else:
        return [t.text for t in nlp.tokenizer(text)]

def process_corpus(corpus, stop_words: set) -> list:
    """Remove capitalizations and stop-words from raw_df, store in long-format DataFrame"""
    token_doc_tups = []
    for document in corpus:
        tokens = tokenize(document.lower())
        tokens_clean = [t.lower() for t in tokens if t.lower() not in stop_words and t.isalpha()]
        token_doc_tups.append(tokens_clean)
    # Just to be sure, remove extra escape characters and punctuations if still
    # present in the words
    token_doc_tups = [' '.join(w for w in line).rstrip('.,\n').lower().split(' ') for line in token_doc_tups]
    return token_doc_tups


def clean_corpus(corpus_raw, freq_threshold=10):
    # Load nltk stop-words
    stop_words = set(stopwords.words('english'))
    # Loop corpus to find uncommon words (less than 10 occurances)
    corpus_freqs = Counter(t.lower() for x in corpus_raw for t in tokenize(x) if t.isalpha())
    uncommon = set([word for (word, freq) in corpus_freqs.items() if freq < freq_threshold])
    # Update stop_words to include uncommon words
    stop_words.update(uncommon)
    # Tokenize corpus and get list of documents
    cleaned_corpus = process_corpus(corpus_raw, stop_words)
    return cleaned_corpus

Select which corpus to analyze.

In [19]:
corpus_name = 'reuters'
# corpus_name = '20newsgroups'

### 20 News Group

In [89]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'),
                shuffle=True, random_state=42)
if corpus_name == '20newsgroups':
    # Process Sklearn news corpus
    cleaned_corpus = clean_corpus(newsgroups_test['data'], freq_threshold=10)

### Reuters

In [90]:
def build_raw_df(corpus, n_files: int) -> pd.DataFrame:
    """Read raw text from corpus files and store in pandas DataFrame"""
    raw_text = []
    for file_id in corpus.fileids()[0:n_files]:
        raw_text.append(corpus.raw(file_id))
    return pd.DataFrame(raw_text, columns=['text'])
if corpus_name == 'reuters':
    # Process Reuters corpus
    cleaned_corpus = clean_corpus(build_raw_df(reuters, n_files=4000)['text'])

In [113]:
corpora = {
    'reuters': clean_corpus(build_raw_df(reuters, n_files=4000)['text']),
    '20newsgroups': clean_corpus(newsgroups_test['data'], freq_threshold=10),
}

### Cleaned Corpus Statistics

In [22]:
# Print some statistics on the corpus size
n_words = 0
for document in cleaned_corpus:
    n_words += len(document)
print(f'Number of words in (tokenized) {corpus_name} corpus: {n_words}')

Number of words in (tokenized) reuters corpus: 274516


In [23]:
for i in range(8):
    print(f'Document n.{i}: {" ".join(word for word in cleaned_corpus[i])}')

Document n.0: asian exporters fear damage mounting trade friction japan raised fears among many asia exporting nations row could far reaching economic damage businessmen officials said told reuter asian move japan might boost protectionist sentiment lead curbs american imports products exporters said conflict would hurt long run short term tokyo loss might gain said impose mln dlrs tariffs imports japanese electronics goods april retaliation japan alleged failure pact sell semiconductors world markets cost japanese estimates put impact tariffs billion dlrs major electronics firms said would virtually halt exports products hit new taxes would able business said spokesman leading japanese electronics firm electric industrial co ltd tariffs remain place time beyond months mean complete exports goods subject tariffs said stock analyst tokyo office broker co taiwan businessmen officials also worried aware threat japan serves warning us said senior trade official asked named taiwan trade tra

## LDA With Collapsed Gibbs Sampling

### NumBa Acceleration

We target two functions for acceleration:
* `initialiaze`: given a document, _i.e._ a matrix of (`n_docs`, `n_words`) containing the indexes of words in the vocabulary, populates the topic and count matrixes
* `sample`: given the aforementioned topic and count matrixes, update conditional distribution, sample the new topics, and finally update the count matrixes.

In [25]:
import numba as nb

@nb.jit(nopython=True)
def sample_topic(n_topics, p):
    # NOTE: The function random.choice with argument p is unsupported in NumBa
    # for nopython compilation. Hence, we rely on an alternative implementation
    # which exploits the cumulative distribution of p and an uniform
    # distribution (which is available in NumBa).
    # return np.random.choice(n_topics, p=p)
    return np.searchsorted(np.cumsum(p), np.random.rand(n_topics))[0]

@nb.jit(forceobj=True, cache=True, parallel=False)
def init_counts(n_topics, n_docs, documents, topics, nmz, nzw, nz, init_topics=False):
    for m in range(n_docs):
        # Retrieve vocab index for i-th word in document m.
        for w in documents[m]:
            # We reached a PAD token, assigned to -1, so we go to next document
            if w < 0:
                continue
            # Get random topic assignment, i.e. z = ...
            if init_topics:
                z = np.random.choice(n_topics) # Uniform distribution at init
            else:
                z = topics[m, w] # pre-initialized at random, just read it
            # Increment count matrices
            nmz[m, z] += 1
            nzw[z, w] += 1
            nz[z] += 1
            if init_topics:
                # Store topic assignment, i.e. topics[(m,i)]=z
                topics[m, w] = int(z)


@nb.jit(nopython=True)
def conditional(alpha, beta, nmz, nzw, nz, n_words, m, w):
    dist = (nmz[m, :] + alpha) * (nzw[:, w] + beta) / (nz + beta * n_words)
    return dist / np.sum(dist)

@nb.jit(nopython=True)
def sample(alpha, beta, n_words, n_docs, n_topics, documents, topics, nmz, nzw, nz):
    for m in range(n_docs):
        # Retrieve vocab index for i-th word in document m.
        for w in documents[m]:
            # We reached a PAD token, assigned to -1, so we go to next document
            if w < 0:
                continue
            # Retrieve topic assignment for i-th word in document m.
            z = topics[m, w]
            # Decrement count matrices.
            nmz[m, z] -= 1
            nzw[z, w] -= 1
            nz[z] -= 1
            # Get conditional distribution.
            p = conditional(alpha, beta, nmz, nzw, nz, n_words, m, w)
            # Sample new topic assignment.
            z = sample_topic(n_topics, p)
            # Increment count matrices.
            nmz[m, z] += 1
            nzw[z, w] += 1
            nz[z] += 1
            # Store new topic assignment.
            topics[m, w] = int(z)

Once defined, the functions are compiled, cached, and tested with random data.

In [26]:
alpha = 0.1
beta = 0.01
n_docs = 3000
n_topics = 10
n_words = 2000
doc_max_len = 1000

documents = np.random.choice(n_words, size=(n_docs, doc_max_len)).astype(np.int32)
topics = np.random.choice(n_topics, size=(n_docs, n_words)).astype(np.int32)
nmz = np.zeros((n_docs, n_topics)).astype(np.int32)
nzw = np.zeros((n_topics, n_words)).astype(np.int32)
nz = np.zeros((n_topics)).astype(np.int32)

init_counts(n_topics, n_docs, documents, topics, nmz, nzw, nz)
sample(alpha, beta, n_words, n_docs, n_topics, documents, topics, nmz, nzw, nz)
sample(alpha, beta, n_words, n_docs, n_topics, documents, topics, nmz, nzw, nz)
sample(alpha, beta, n_words, n_docs, n_topics, documents, topics, nmz, nzw, nz)

np.allclose(nmz, np.zeros((n_docs, n_topics)).astype(np.int32))

<ipython-input-25-58c58fc7d4f7>:12: NumbaWarning: Cannot cache compiled function "init_counts" as it uses lifted code
  @nb.jit(forceobj=True, cache=True, parallel=False)


False

### LDA Class

In [111]:
class LDACGS:
    """Do LDA with Collapsed Gibbs Sampling."""
    def __init__(self, n_topics=2, alpha=0.1, beta=0.01, corpus=None):
        """Initialize system parameters."""
        self.n_topics = n_topics
        self.alpha = alpha
        self.beta = beta
        self.vocab_built = False
        if corpus:
            self.buildVocabulary(corpus)
            self.vocab_built = True

    def buildVocabulary(self, corpus):
        """Given a corpus as a list of list of words, build the vocabulary."""
        self.corpus = corpus
        n_docs = len(corpus)
        self.vocab = list({v for doc in corpus for v in doc})
        # ======================================================================
        # Transform the words in the corpus to indexes to the words in the
        # vocabulary
        # ======================================================================
        self.documents = []
        self.doc_max_len = 0
        for i in range(n_docs):
            self.documents.append({})
            for j in range(len(corpus[i])):
                if corpus[i][j] in self.vocab:
                    self.documents[i][j] = self.vocab.index(corpus[i][j])
            if len(self.documents[i]) > self.doc_max_len:
                self.doc_max_len = len(self.documents[i])
        print(f'INFO. Number of documents in corpus: {len(self.documents)}')
        print(f'INFO. Vocabulary size: {len(self.vocab)}')
        self.vocab_built = True

    def initialize(self, n_topics=None):
        """Initialize the three count matrices."""
        if n_topics:
            self.n_topics = n_topics    
        self.n_words = len(self.vocab)
        self.n_docs = len(self.documents)
        # ======================================================================
        # Convert list of dictionaries to pure numpy arrays
        # ======================================================================
        # NOTE: Init document matrix with PAD tokens, i.e. -1
        self.documents_np = -np.ones((self.n_docs, self.doc_max_len),
                                     dtype=np.int32)
        for i in range(self.n_docs):
            vals = list(self.documents[i].values())
            for j in range(len(vals)):
                self.documents_np[i][j] = int(vals[j])
        # ======================================================================
        # Instantiate the three count matrices
        # ======================================================================
        # The (i, j) entry of self.nmz is the number of words in document i
        # assigned to topic j.
        self.nmz = np.zeros((self.n_docs, self.n_topics), dtype=np.int32)
        # The (i, j) entry of self.nzw is the number of times word j is
        # assigned to topic i.
        self.nzw = np.zeros((self.n_topics, self.n_words), dtype=np.int32)
        # The (i)-th entry is the number of times topic i is assigned in the
        # corpus.
        self.nz = np.zeros(self.n_topics, dtype=np.int32)
        # Initialize the topic assignment matrix to uniform random topics
        sz = (self.n_docs, self.n_words)
        self.topics = np.random.choice(self.n_topics, size=sz).astype(np.int32)
        # ======================================================================
        # Initialize the topics and the three count matrices
        # ======================================================================
        init_counts(self.n_topics, self.n_docs, self.documents_np, self.topics,
                    self.nmz, self.nzw, self.nz)

    def sample(self, corpus=None, burnin=100, n_topics=None, alpha=None,
               beta=None, verbose=0):
        """Run collapsed Gibbs sampling iterations"""
        if alpha:
            self.alpha = alpha
        if beta:
            self.beta = beta
        if not self.vocab_built:
            if verbose:
                print(f'INFO. Building corpus.')
            self.buildVocabulary(corpus)
        if verbose:
            print(f'INFO. Vocabulary built. Initializing.')
        # ======================================================================
        # Initialize count matrices
        # ======================================================================
        self.initialize(n_topics)
        # ======================================================================
        # Perform sampling
        # ======================================================================
        if verbose:
            print(f'INFO. Starting Burn-in sampling.')
        for i in tqdm(range(burnin)):
            sample(self.alpha, self.beta, self.n_words, self.n_docs,
                  self.n_topics, self.documents_np, self.topics, self.nmz,
                  self.nzw, self.nz)
        # ======================================================================
        # After discarding the initial samples, make one actual sampling
        # ======================================================================
        sample(self.alpha, self.beta, self.n_words, self.n_docs,
                self.n_topics, self.documents_np, self.topics, self.nmz,
                self.nzw, self.nz)
    
    def D(self, widx, cowidx=None):
        """
        Get the number of documents in which the word index appears at least
        once

        Parameters
        ----------
        widx : int
            Vocabulary index of the word to count
        cowidx : int, optional
            Vocabulary index of the co-word to count

        Returns
        -------
        int
            the number of documents in which the word index appears at least
            once, if coidx is not supplied, else the number of documents in
            which they both appear together
        """
        matches = np.bitwise_or.reduce(self.documents_np == widx, axis=1)
        if cowidx:
            comatches = np.bitwise_or.reduce(self.documents_np == cowidx, axis=1)
            return np.logical_and(matches, comatches).sum()
        else:
            return matches.sum()
    
    def Umass(self, topics):
        C = np.zeros(len(topics))
        for i, topic_words in enumerate(topics):
            for vm, vl in itertools.combinations(topic_words, r=2):
                C[i] += np.log((self.D(vm, vl) + 1) / self.D(vm))
        return C
    
    def getTopTerms(self, n_terms=10):
        """Get the list of the top-n_terms words per topic"""
        # ======================================================================
        # Get topics distribution Phi, with shape: (n_topics, n_words)
        # ======================================================================
        phi = self.nzw + self.beta
        phi = phi / (np.sum(phi, axis=1)[:, np.newaxis])
        # ======================================================================
        # Sort probabilities per topic, i.e. row, and get the ordered word
        # indexes in the vocabulary
        # ======================================================================
        idx = np.argsort(phi, axis=1).astype(np.int32)
        # ======================================================================
        # Finally, select the top-N words for each fo the topics
        # ======================================================================
        topics, topics_idx = [], []
        for k in range(self.n_topics):
            topics.append([self.vocab[idx[k, self.n_words - 1 - i]] for i in range(n_terms)])
            topics_idx.append([idx[k, n_words - 1 - i] for i in range(n_terms)])
        return topics, self.Umass(topics_idx)

In [85]:
ldacgs = LDACGS(corpus=cleaned_corpus)

INFO. Number of documents in corpus: 4000
INFO. Vocabulary size: 3643


In [86]:
beta = 0.1 # 0.1 or 0.01
n_topics = 10 # 10 or 50
burnin = 100 # 100 or 200
ldacgs.sample(burnin=burnin, beta=beta, n_topics=n_topics, verbose=0)

100%|██████████| 200/200 [00:50<00:00,  3.96it/s]


In [87]:
topics, u_mass = ldacgs.getTopTerms(n_terms=10)
for i, topic in enumerate(topics):
    print(f'Topic n.{i}: {", ".join(t for t in topic)}')
print(f'Umass coherence score: {u_mass.mean()}')

Topic n.0: said, year, would, two, today, profit, expected, february, major, tonnes
Topic n.1: net, group, shares, told, agreement, international, total, offer, four, japan
Topic n.2: may, nine, april, price, march, prices, end, operations, business, higher
Topic n.3: also, loss, government, exchange, increase, month, current, rate, securities, agreed
Topic n.4: share, first, five, oil, six, record, due, earnings, since, sale
Topic n.5: mln, shr, company, revs, shrs, ltd, interest, tax, unit, says
Topic n.6: cts, market, bank, week, made, ended, assets, cash, shareholders, chairman
Topic n.7: inc, qtr, corp, note, last, mths, co, per, avg, june
Topic n.8: dlrs, billion, one, stock, trade, could, months, years, buy, rise
Topic n.9: vs, pct, sales, new, three, dlr, added, foreign, president, statement
Umass coherence score: -186.54882996969266


## $U_{MASS}$ Coherence Score

The $U_{MASS}$ coherence score is defined as follows:

\begin{equation}
    C(t; V^{(t)})= \sum_{m=2}^M \sum_{l=1}^{m-1} \log \frac{\mathbf{D}\big(v_m^{(t)}, v_l^{(t)}\big) + 1}{\mathbf{D}\big(v_l^{(t)}\big)}
\end{equation}

where $\mathbf{D}\big(v_m^{(t)},\, v_l^{(t)}\big)$ represents the number of documents $d$ which contain at least one each of the words $v_m$ and $v_l$ for topic $t$.

The following implementations have been later embedded in the `LDACGS` class.

In [ ]:
import itertools

beta = ldacgs.beta
n_topics = ldacgs.n_topics
vocab = ldacgs.vocab
documents = ldacgs.documents_np
nzw = ldacgs.nzw

def D(widx, documents, cowidx=None):
    """Get the number of documents in which the word index appears at least once

    Parameters
    ----------
    widx : int
        Vocabulary index of the word to count
    documents : numpy array of int32
        Corpus containing words as vocabulary indexes, shape: (n_docs, max_doc_len)
    cowidx : int, optional
        Vocabulary index of the co-word to count

    Returns
    -------
    int
        the number of documents in which the word index appears at least once,
        if coidx is not supplied, else the number of documents in which they
        both appear together
    """
    matches = np.bitwise_or.reduce(documents == widx, axis=1)
    if cowidx:
        comatches = np.bitwise_or.reduce(documents == cowidx, axis=1)
        return np.logical_and(matches, comatches).sum()
    else:
        return matches.sum()


def get_top_topics(nzw, vocab, n_terms=10, beta=0.1):
    n_topics = nzw.shape[0]
    n_words = len(vocab)
    # Get topics distribution Phi, with shape: (n_topics, n_words)
    phi = nzw + beta
    phi = phi / (np.sum(phi, axis=1)[:, np.newaxis])
    # Sort probabilities and get the ordered word indexes in the vocabulary
    widx = np.argsort(phi, axis=1).astype(np.int32)
    # Finally, select the top-N words for each fo the topics
    topics, topics_idx = [], []
    for k in range(n_topics):
        topics.append([vocab[widx[k, n_words - 1 - i]] for i in range(n_terms)])
        topics_idx.append([widx[k, n_words - 1 - i] for i in range(n_terms)])
    return topics, topics_idx

topics, topics_idx = get_top_topics(nzw, vocab, 10, beta)
for i, topic in enumerate(topics):
    print(f'Topic n.{i}: {", ".join(t for t in topic)}')


@nb.jit(nopython=True)
def combinations(pool, r):
    n = len(pool)
    indices = list(range(r))
    empty = not(n and (0 < r <= n))
    if not empty:
        result = [pool[i] for i in indices]
        yield result
    while not empty:
        i = r - 1
        while i >= 0 and indices[i] == i + n - r:
            i -= 1
        if i < 0:
            empty = True
        else:
            indices[i] += 1
            for j in range(i+1, r):
                indices[j] = indices[j-1] + 1
            result = [pool[i] for i in indices]
            yield result


def Umass(topics, documents):
    C = np.zeros(len(topics))
    for i, topic_words in enumerate(topics):
        for vm, vl in itertools.combinations(topic_words, r=2):
            C[i] += np.log((D(vm, documents, vl) + 1) / D(vm, documents))
    return C

u_mass = Umass(np.array(topics_idx), documents)
print(f'Umass coherence score: {u_mass.mean()}')

## Evaluation

In [114]:
table_data = []
for corpus_name in corpora.keys():
    # Get previously tokenized and cleaned corpus
    cleaned_corpus = corpora[corpus_name]
    # Instantiate LDACGS, i.e. build the corpus vocabulary
    ldacgs = LDACGS(corpus=cleaned_corpus)
    # Evaluate all combinations
    for K in [10, 50]:
        for burnin in [100, 200]:
            for alpha, beta in [(0.1, 0.1), (0.1, 0.01)]:
                # Perform Gibbs sampling
                ldacgs.sample(burnin=burnin, n_topics=K, alpha=alpha, beta=beta)
                # Get topics
                topics, u_mass = ldacgs.getTopTerms(n_terms=20)
                # Dump topics and Umass per topic to file
                topicfile = f'topics_{corpus_name}_{K}_{burnin}_{alpha}_{beta}.txt'
                with open(topicfile, 'w') as f:
                    for i, topic in enumerate(topics):
                        f.write(f'Topic n.{i} (Umass = {u_mass[i]:.3f}): {", ".join(t for t in topic)}\n')
                # Populate table entries
                print(f'Umass coherence score for {K} topics and burn-in {burnin} (α={alpha}, β={beta}): {u_mass.mean():.3f}')
                table_data.append((corpus_name, K, burnin, alpha, beta, round(u_mass.mean(), 3)))
cols = ['Corpus', 'N. Topics', 'Burn-in', 'Alpha', 'Beta', 'Umass Score']
table = pd.DataFrame(table_data, columns=cols)
table.to_csv(f'collapsed_gibbs_results.csv', encoding='utf-8', index=False)

INFO. Number of documents in corpus: 4000
INFO. Vocabulary size: 3643


100%|██████████| 100/100 [00:25<00:00,  3.85it/s]


Umass coherence score for 10 topics and burn-in 100 (α=0.1, β=0.1): -504.108


100%|██████████| 100/100 [00:24<00:00,  4.01it/s]


Umass coherence score for 10 topics and burn-in 100 (α=0.1, β=0.01): -516.453


...

Umass coherence score for 50 topics and burn-in 100 (α=0.1, β=0.01): -488.088


100%|██████████| 200/200 [05:21<00:00,  1.61s/it]


Umass coherence score for 50 topics and burn-in 200 (α=0.1, β=0.1): -482.744


100%|██████████| 200/200 [05:17<00:00,  1.59s/it]


Umass coherence score for 50 topics and burn-in 200 (α=0.1, β=0.01): -483.105


In [115]:
table

,Corpus,N. Topics,Burn-in,Alpha,Beta,Umass Score
0,reuters,10,100,0.1,0.10,-504.108
1,reuters,10,100,0.1,0.01,-516.453
2,reuters,10,200,0.1,0.10,-502.423
3,reuters,10,200,0.1,0.01,-507.965
4,reuters,50,100,0.1,0.10,-558.461
5,reuters,50,100,0.1,0.01,-627.397
6,reuters,50,200,0.1,0.10,-534.134
7,reuters,50,200,0.1,0.01,-616.861
8,20newsgroups,10,100,0.1,0.10,-503.679
9,20newsgroups,10,100,0.1,0.01,-516.199


In [116]:
!zip collapsed_gibbs_results.zip collapsed_gibbs_results.csv topics_*

  adding: collapsed_gibbs_results.csv (deflated 66%)
  adding: topics_20newsgroups_10_100_0.1_0.01.txt (deflated 55%)
  adding: topics_20newsgroups_10_100_0.1_0.1.txt (deflated 55%)
  adding: topics_20newsgroups_10_200_0.1_0.01.txt (deflated 55%)
  adding: topics_20newsgroups_10_200_0.1_0.1.txt (deflated 55%)
  adding: topics_20newsgroups_50_100_0.1_0.01.txt (deflated 60%)
  adding: topics_20newsgroups_50_100_0.1_0.1.txt (deflated 60%)
  adding: topics_20newsgroups_50_200_0.1_0.01.txt (deflated 60%)
  adding: topics_20newsgroups_50_200_0.1_0.1.txt (deflated 60%)
  adding: topics_reuters_10_100_0.1_0.01.txt (deflated 55%)
  adding: topics_reuters_10_100_0.1_0.1.txt (deflated 54%)
  adding: topics_reuters_10_200_0.1_0.01.txt (deflated 54%)
  adding: topics_reuters_10_200_0.1_0.1.txt (deflated 54%)
  adding: topics_reuters_50_100_0.1_0.01.txt (deflated 61%)
  adding: topics_reuters_50_100_0.1_0.1.txt (deflated 61%)
  adding: topics_reuters_50_200_0.1_0.01.txt (deflated 61%)
  adding: topi

In [107]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [117]:
import os

drive_dir = os.path.join(os.path.abspath(''), 'drive', 'MyDrive')
!cp collapsed_gibbs_results.zip $drive_dir

## Gibbs Sampling: Alternative 1

In [ ]:
# PARAMETERS: alpha, beta, K, D, p
# (where p is the Cat(p1, p2, ..., p_K) distribution initialized uniformly)
alpha = 0.1
beta = 0.1
K = 50

# Build vocabulary
vocab = list({v for doc in cleaned_corpus for v in doc})
vocab.sort()
# Storing the topic assigned to each word in the corpus
# format:
#   row 1: document ID (index)
#   row 2: word ID (w/in the vocab)
#   row 3: word topic k
topic_mat = []
for d, doc in enumerate(cleaned_corpus):
    for word in doc:
        topic_mat.append((d, vocab.index(word), int(np.random.choice(a=K))))
topic_mat = np.array(topic_mat, dtype=np.int32).T

# Define Numpy count matrices for n and m
D = len(cleaned_corpus) + 1 # Number of documents
V = len(vocab)
n_d = np.zeros((D, K), dtype=np.int32)
m_k = np.zeros((V, K), dtype=np.int32)

topic_mat.shape

# W = len(df) # Total number of words in the corpus
# V = len(df.word_id.unique()) # Number of words in the vocabulary
# D = df.document.max() + 1 # Number of documents
# # Initialize topics k randomly from p
# df['z'] = np.random.choice(a=K, size=W, p=p)
# # Storing the topic assigned to each word in the corpus
# # format:
# #   row 1: document ID (index)
# #   row 2: word ID (w/in the vocab)
# #   row 3: word topic k
# topic_mat = df[['document', 'word_id', 'z']].to_numpy(dtype=np.int32).T

In [ ]:
from numba import jit, float32

"""
Calculating n_d and m_k:
Given topic_mat : 
    - row 0 == indicators for document 
    - row 1 == indicators for word (word_id)
    - row 2 == z, i.e. k-value for each word

n_d implementation :
    vectorize search for document + topic matches in document
    (i.e unique row 0 + row 2 matches)
    let position n_d[doc, topic] == # positions in doc d with topic k

m_k implementation :
    vectorize search for word + topic matches in corpus 
    (i.e unique row 1 + row 2 matches)
    let position m_k[word, topic] == # positions in corpus with
    topic k and word w
"""
@jit(forceobj=True, cache=True)
def init_count_mat(topic_mat, n_d, m_k):
    # Populate n and m 
    # n_d
    vals, counts = np.unique(topic_mat[[0, 2], :], return_counts=True, axis=1)
    n_d[vals[0], vals[1]] = counts
    # m_k
    val, count = np.unique(topic_mat[[1, 2], :], return_counts=True, axis=1)
    m_k[val[0], val[1]] = count

# @jit(nopython=True, cache=True)
def draw_from_prob_matrix(prob_matrix, axis=1):
    """
    With this implementation we get probability matrix p with shape (V, K).
    The function lets us sample by drawing one value from each distribution
    according to the distribution at row w_dj.
    """
    r = np.expand_dims(np.random.rand(prob_matrix.shape[1-axis]), axis=axis)
    return (prob_matrix.cumsum(axis=axis) > r).argmax(axis=axis)

In [ ]:
# @jit(nopython=True, cache=True, parallel=False)
def sample_alt1(n_iter, topic_mat, n_d, m_k):
    DOCS = 0
    WORDS = 1
    TOPICS = 2
    # Gibbs sampling with n_iter iterations
    for i in tqdm(range(n_iter)):
        # Update probabilities and draw samples for k
        p = n_d[topic_mat[DOCS]] * m_k[topic_mat[WORDS], :]
        p = p / p.sum(axis=WORDS, keepdims=True)
        topic_mat[TOPICS] = draw_from_prob_matrix(p)
        # Update n_d
        n_vals, n_counts = np.unique(topic_mat[[DOCS, TOPICS], :], return_counts=True, axis=WORDS)
        n_d[n_vals[DOCS], n_vals[WORDS]] = n_counts
        # Update m_k
        m_vals, m_counts = np.unique(topic_mat[WORDS:, :], return_counts=True, axis=WORDS)
        m_k[m_vals[DOCS], m_vals[WORDS]] = m_counts
    p = n_d[topic_mat[DOCS]] * m_k[topic_mat[WORDS], :]
    return p / p.sum(axis=WORDS, keepdims=True)

In [ ]:
n_iter = 200
init_count_mat(topic_mat, n_d, m_k)
z = sample_alt1(n_iter, topic_mat, n_d, m_k)

100%|██████████| 200/200 [01:49<00:00,  1.83it/s]


In [ ]:
print(topic_mat.shape)
print(z.shape)

(3, 274494)
(274494, 50)
